In [ ]:
# install some packages

# for easy downloading from google drive
!pip install gdown 

# for loading parquet
!pip install pyarrow
!pip install fastparquet

# for models
!pip install pytorch_lightning
!pip install pytorch-forecasting

In [ ]:
# setup Kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir /root/.kaggle
import json
token = {"username":"yoffoy","key":"????"}
!pwd
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
for i in range(2):
    !chmod 600 /root/.kaggle/kaggle.json
    !cp /root/.kaggle/kaggle.json ~/.kaggle/
    !kaggle config set -n path -v /root
    !cat /root/.kaggle/kaggle.json

     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=4dd5ae390e2d210c199fd8efd927933ecb2a6aa5ffe626e69420614f6831d787
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
/content
cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root
{
  "username": "yoffoy",
  "key": "28775e476cb4bc85480d4b20b6bb47a4",
  "path": "/root"
}cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root
{
  "username": "yoffoy",
  "key": "28775e476cb4bc85480d4b20b6bb47a4",
  "path": "/root"
}

In [ ]:
import os
import sys
import pickle


import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as ttf

import os
import os.path as osp

from tqdm import tqdm
from sklearn.metrics import roc_auc_score


# Download Data & Create Test DataFrame

In [ ]:
if True: # download unprocessed data in parquet form
    !kaggle datasets download -d robikscube/ubiquant-parquet -p /content/ubiquant
    !unzip -q /content/ubiquant/ubiquant-parquet.zip -d /content/ubiquant

100% 12.5G/12.5G [02:03<00:00, 130MB/s]
100% 12.5G/12.5G [02:03<00:00, 109MB/s]


In [ ]:
df = pd.read_parquet('/content/ubiquant/train_low_mem.parquet')


In [ ]:
!gdown 1QBvxnH4Jb_wZ5lMf1ATSssEwo3UTQmQn -O /content/ubiquant/test_500_ids.pkl

with open('/content/ubiquant/test_500_ids.pkl', 'rb') as f:
    test_500_ids = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?id=1QBvxnH4Jb_wZ5lMf1ATSssEwo3UTQmQn
To: /content/ubiquant/test_500_ids.pkl
100% 1.49k/1.49k [00:00<00:00, 2.36MB/s]


In [ ]:
test_df = df[(df['investment_id'].isin(test_500_ids)) & (df['time_id'] >= 1100)]

In [ ]:
test_df.investment_id = test_df.investment_id.astype(str)
test_df.time_id = test_df.time_id.astype(int)
test_df.dtypes

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


row_id            object
time_id            int64
investment_id     object
target           float32
f_0              float32
                  ...   
f_295            float32
f_296            float32
f_297            float32
f_298            float32
f_299            float32
Length: 304, dtype: object

In [ ]:
test_df

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
2743821,1100_4,1100,4,2.001431,-1.856335,-0.534634,0.812365,-0.181939,0.266184,2.398910,...,-0.558287,0.058620,-1.681110,0.154686,-1.041528,0.216318,1.526667,0.868935,-0.159243,0.143675
2743831,1100_16,1100,16,2.562713,-0.647350,1.246443,0.256431,-0.122367,-0.094262,-1.953241,...,2.193173,-1.497249,-0.106696,-2.601423,-0.522216,-0.296990,1.375808,2.115156,0.036839,0.309283
2743839,1100_28,1100,28,-0.046583,2.277458,-0.297993,0.118446,-0.160365,-0.146017,1.389627,...,0.219786,-1.378826,-0.566242,0.131817,1.997294,-0.634687,-1.269228,-1.331287,-0.424596,-0.588568
2743840,1100_29,1100,29,0.004562,0.156907,-0.604645,-0.037411,-0.079290,-0.127519,1.492638,...,1.689814,-0.439729,-0.273137,1.746683,0.396428,-0.288306,0.700139,2.522879,-1.392536,-0.246457
2743844,1100_34,1100,34,-0.141138,1.119750,-2.016626,0.703738,-0.210108,0.028373,-3.569144,...,-1.495973,-3.002694,-0.198247,1.366903,3.095617,0.251274,-0.100225,-0.476354,-1.576642,0.431461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141351,1219_3711,1219,3711,-1.187621,0.995189,0.190229,-0.927608,1.545319,-0.176870,0.420421,...,-0.317095,0.811402,2.279778,0.875537,0.421628,0.288127,-0.075548,0.136557,-0.587762,-0.733876
3141358,1219_3719,1219,3719,-0.580605,-0.228733,1.751094,-2.355600,1.359468,0.119455,0.015525,...,-0.803114,0.811402,-1.026077,0.875537,-2.371759,0.125203,-0.075548,2.065139,-0.769788,0.199461
3141369,1219_3730,1219,3730,0.365068,0.338434,-0.460131,1.522161,-0.456760,0.006015,-0.460537,...,0.654942,0.811402,-0.909691,-1.142157,0.421628,0.002889,-0.075548,-0.128885,0.863475,0.093495
3141395,1219_3757,1219,3757,1.567879,-1.848767,1.230805,0.181467,1.399093,0.049054,0.616560,...,0.137736,-1.232434,-0.337345,0.875537,-2.371759,0.091494,-0.075548,0.853875,-1.416803,0.653864


# Load Model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning.loggers import WandbLogger


In [ ]:
# Download Trained Model
! gdown 18cTuDxN43rrvlWsMU7BsCc92HMJ812C- -O /content/trained_model/tft_baseline_426.pth

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 166, in main
    resume=args.continue_,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 202, in download
    for file in os.listdir(osp.dirname(output) or "."):
FileNotFoundError: [Errno 2] No such file or directory: '/content/trained_model'


In [ ]:
# Download Dataset used for training
! gdown 1SZAQ7_BC3ODEf5l7l1bVYpoEtJC3i4_l -O /content/pf_train_1000_samples.pf

Downloading...
From: https://drive.google.com/uc?id=1SZAQ7_BC3ODEf5l7l1bVYpoEtJC3i4_l
To: /content/pf_train_1000_samples.pf
100% 1.14G/1.14G [00:11<00:00, 102MB/s]


In [ ]:
# args taken from Han Li

NUM_WORKERS = 4 # Use 4 for AWS


# Baseline hyperparameters: 
# TFT_quantile_loss_tune_10
# https://wandb.ai/11785_project/11785_project_tuning
args = {
    # ------------------------------
    # Basic config
    'random_seed': 11785,
    'n_samples': 1000,
    'batch_size': 64,
    'n_workers' : NUM_WORKERS,
    'criterion': {
        'quantile': QuantileLoss(),
#        'pearson': Pearson.Pearson(),   # Miao's implementation
        'other': None,                  # TODO: check out other loss (e.g., MSE)
    },
    # ------------------------------
    # Hyperparameters
    'lr_s': 7e-3,
    'hidden_size': 512,
    'attention_head_size': 2,        # use multihead for large hidden size
    'dropout': 0.1,
    'hidden_continuous_size': 4,     # set to <= hidden_size
    'output_size': 7,                # 7 quantiles for QuantileLoss by default
    'reduce_on_plateau_patience': 4, # reduce learning rate if no improvement in validation loss after x epochs
    'gradient_clip_val': 0.1,
    # ------------------------------
    # Logging
    'log_interval': 5,               # log every n batches, set to None when try to find best lr
    'wandb_entity': '11785_project',
    'wandb_project': '11785_project_tuning',
    'wandb_name': 'TFT_baseline_0426',
}

In [ ]:
n = args['n_samples']

train_dataset = TimeSeriesDataSet.load("/content/pf_train_1000_samples.pf")

train_dataloader = train_dataset.to_dataloader(train=True, batch_size=args['batch_size'], num_workers=args['n_workers'])

print("Load existing datasets completed:")
print(f"Train dataset:  pf_train_{n}_samples.pf")

AttributeError: ignored

In [ ]:
tft_model = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=args['lr_s'],
    hidden_size=args['hidden_size'],  # most important hyperparameter apart from learning rate
    attention_head_size=args['attention_head_size'], # number of attention heads. Set to up to 4 for large datasets
    dropout=args['dropout'],  # between 0.1 and 0.3 are good values
    hidden_continuous_size=args['hidden_continuous_size'],
    output_size=args['output_size'],
    loss=args['criterion']['quantile'],
    # loss=args['criterion']['pearson'],
    log_interval=args['log_interval'],  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=args['reduce_on_plateau_patience'], # reduce learning rate if no improvement in validation loss after x epochs
)

tft_model.load_state_dict(torch.load('/content/trained_model/tft_baseline_426.pth'))
tft_model.eval()

row_id            object
time_id           uint16
investment_id     uint16
target           float32
f_0              float32
                  ...   
f_295            float32
f_296            float32
f_297            float32
f_298            float32
f_299            float32
Length: 304, dtype: object

# Test

I use data with time_id >= 1000 for testing. 
It turns out that there are 120 different time_id's for test data. 
I divde them into 6 parts: 0-19, 0-39, 0-59, 0-79, 0-99, 0-119.
Then run predictions on them.

I use encoder length = 14, decoder length = 3. 
Per my understanding, the model will tune itself on previous data and then make predictions on the last 3 time steps for each investment id. 
For example, take 0-59. The model will tune itself on 0-42 (not sure about this, it seems that it does make use of these data, but I'm not sure how), encode 43-56, and predict 57-59.  

#### Create Test DataFrames

In [ ]:
test_dfs = dict()
for i in range(6):
    test_dfs['test_df_' + str(i)] = test_df.head(math.floor(((i+1)/6) *len(test_df. index)))
    
print(test_dfs)

# Test using correlation

In [ ]:
errors = list()

for i in range(6):
    # create test_dataset from test_dfs, with parameter from train_dataset
    test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, test_dfs['test_df_' + str(i)], predict=True, stop_randomization=True, allow_missing_timesteps=True)
    test_dataloader = test_dataset.to_dataloader(train=False, batch_size=64, num_workers=4)

    # make predictions
    predictions, inputs = tft_model.predict(test_dataloader, mode='prediction', return_x=True)

    # calculate correlation 
    y1 = predictions.reshape(1, -1)
    t1 = inputs['decoder_target'].reshape(1, -1)
    z = torch.cat((y1, t1), dim=0) # 2, B*T+
    error = torch.corrcoef(z)[0][1]

    errors.append(error)


In [ ]:
print(errors)